In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, classification_report
import numpy as np

# Helper function to compute password entropy
def compute_entropy(password):
    entropy = len(set(password)) / len(password)
    return entropy

# Load and preprocess password dataset
df = pd.read_csv('password_data.csv')
df['length'] = df['password'].apply(len)
df['has_upper'] = df['password'].apply(lambda x: any(c.isupper() for c in x)).astype(int)
df['has_lower'] = df['password'].apply(lambda x: any(c.islower() for c in x)).astype(int)
df['has_special'] = df['password'].apply(lambda x: any(not c.isalnum() for c in x)).astype(int)
df['entropy'] = df['password'].apply(compute_entropy)
X = df[['length', 'has_upper', 'has_lower', 'has_special', 'entropy']]
y = df['strength']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Model training and cross-validation
model = RandomForestClassifier(n_estimators=100)
cross_val_scores = cross_val_score(model, X_train, y_train, cv=5)
print("Cross-validation scores:", cross_val_scores)

# Fit model and evaluate performance
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print("Password Strength Classification Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

# Real-time prediction function
def predict_password_strength(password):
    features = {
        'length': len(password),
        'has_upper': int(any(c.isupper() for c in password)),
        'has_lower': int(any(c.islower() for c in password)),
        'has_special': int(any(not c.isalnum() for c in password)),
        'entropy': compute_entropy(password)
    }
    feature_vector = np.array([list(features.values())]).reshape(1, -1)
    strength_prediction = model.predict(feature_vector)
    return strength_prediction[0]

# Example usage
print("Password 'Secure@123' strength:", predict_password_strength("Secure@123"))

# Save model
import joblib
joblib.dump(model, 'password_strength_model.pkl')
print("Model saved as password_strength_model.pkl")